In [14]:
import requests
import os

data_url = "https://raw.githubusercontent.com/jamescalam/data/main/sentence_embeddings_15K/"


# create data directory to store data
if not os.path.exists('./data'):
    os.mkdir('./data')

# download the numpy binary files (dense vectors)
for i in range(57):
    res = requests.get(data_url+f"embeddings_{i}.npy")
    with open(f'./data/embeddings_{i}.npy', 'wb') as fp:
        for chunk in res:
            fp.write(chunk)
    print('.', end='')

.........................................................

In [7]:
# and download the respective text file
res = requests.get(f"{data_url}sentences.txt")
with open(f"./data/sentences.txt", 'wb') as fp:
    for chunk in res:
        fp.write(chunk)

In [15]:
import numpy as np

In [16]:
for i in range(57):
    if i == 0:
        with open(f'./data/embeddings_{i}.npy', 'rb') as fp:
            sentence_embeddins = np.load(fp)
    else:
        with open(f'./data/embeddings_{i}.npy', 'rb') as fp:
            sentence_embeddins = np.append(sentence_embeddins, np.load(fp), axis=0)

In [17]:
sentence_embeddins.shape

(14504, 768)

In [18]:
with open('./data/sentences.txt', 'r') as fp:
    lines = fp.read().split('\n')

In [19]:
lines[:5]

['A group of four children dancing in a backyard.',
 'The Conference Board said its measure of business confidence, which had fallen to 53 in the first quarter of 2003, improved to 60 in the most recent second quarter.',
 'a person eating a meal, often in a restaurant',
 'When you crossed the line, you violated the constitutional right," said Charles Weisselberg, who teaches law at the University of California, Berkeley.',
 "Ross Garber, Rowland's legal counsel, said the governor would have no comment on the condo deal."]

In [20]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [21]:
d = sentence_embeddins.shape[1]

In [22]:
# IndexFlatL2

import faiss

index = faiss.IndexFlatL2(d)

In [23]:
index.is_trained

True

In [24]:
index.add(sentence_embeddins)

In [25]:
index.ntotal

14504

In [26]:
xq = model.encode(['someone aprints with a football'])

In [27]:
# similer vector count
k = 4

In [28]:
%%time
D, I = index.search(xq, k)
print(I)

[[12465  9596  8655  9823]]
CPU times: user 238 ms, sys: 0 ns, total: 238 ms
Wall time: 39.5 ms


In [29]:
[f'{i}: {lines[i]}' for i in I[0]]

['12465: Two groups of people are playing football',
 '9596: Football players are on the field.',
 '8655: position in football played by a team member',
 '9823: A man with a shirt is holding a football']

Index 추가하기

In [30]:
#셀 정의
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [31]:
index.is_trained

False

In [32]:
index.train(sentence_embeddins)

In [33]:
index.is_trained

True

In [34]:
index.add(sentence_embeddins)
index.ntotal

14504

In [35]:
%%time
D, I = index.search(xq, k)
print(I)

[[12465  9596  8655  9823]]
CPU times: user 46.3 ms, sys: 3.42 ms, total: 49.7 ms
Wall time: 7.71 ms


In [36]:
[f'{i}: {lines[i]}' for i in I[0]]

['12465: Two groups of people are playing football',
 '9596: Football players are on the field.',
 '8655: position in football played by a team member',
 '9823: A man with a shirt is holding a football']

In [37]:
# 검색값이 그다지 정확하지 않을 경우
index.nprobe = 10

In [38]:
%%time
D, I = index.search(xq, k)
print(I)

[[12465  9596  8655  9823]]
CPU times: user 25.2 ms, sys: 0 ns, total: 25.2 ms
Wall time: 3.36 ms


In [39]:
m = 8 # m은 d 의 배수여야함, 벡터 클러스터의 중심
bits = 8

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [40]:
index.is_trained

False

In [41]:
index.train(sentence_embeddins)

In [42]:
index.add(sentence_embeddins)

In [43]:
%%time
D, I = index.search(xq, k)
print(I)

[[  399  8328 12465   190]]
CPU times: user 15.7 ms, sys: 3.57 ms, total: 19.3 ms
Wall time: 2.8 ms


In [44]:
[f'{i}: {lines[i]}' for i in I[0]]
# 속도와 정확성은 trade off

['399: A football player kicks the ball.',
 '8328: A football player is running past an official carrying a football',
 '12465: Two groups of people are playing football',
 '190: A person playing football is running past an official carrying a football']